<a href="https://colab.research.google.com/github/PrajwalSolanki/basic/blob/main/impersonator_plus_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Note
Make sure that your runtime type is 'Python 3.6+ with GPU acceleration'. To do so, go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

# Impersonator++

All Resources:

[![GitHub stars](https://img.shields.io/github/stars/iPERDance/iPERCore?style=social)](https://github.com/iPERDance/iPERCore)

- Paper: https://arxiv.org/pdf/2011.09055.pdf
- Repo: https://github.com/iPERDance/iPERCore
- Project Page: https://www.impersonator.org/work/impersonator-plus-plus.html
- Dataset https://svip-lab.github.io/dataset/iPER_dataset.html
- Forum https://discuss.impersonator.org/




In [ ]:
#%%

import IPython
IPython.display.HTML(
   '<h2>Demo</h2><iframe width="75%" height="512" src="https://www.impersonator.org/project_img/impersonator_plus_plus/demo_video/demo_1_512x512.mp4" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
)


# Dependencies

## System Requirements
 - Linux (test on Ubuntu 16.04 and 18.04) or Windows (test on windows 10)
 - CUDA 10.1, 10.2, or 11.0
 - gcc 7.5+ (needs to support C++14)
 - ffmpeg (ffprobe) 4.3.1+

## Python Requirements
- Python 3.6+
- PyTorch tested on 1.7.0
- Torchvison tested on 0.8.1
- mmcv-full test on 1.2.0
- numpy>=1.19.3
- scipy>=1.5.2
- scikit-image>=0.17.2
- opencv-python>=4.4.0.40
- tensorboardX>=2.1
- tqdm>=4.48.2
- visdom>=0.1.8.9
- easydict>=1.9
- toml>=0.10.2
- git+https://github.com/open-mmlab/mmdetection.git@8179440ec5f75fe95484854af61ce6f6279f3bbc
- git+https://github.com/open-mmlab/mmediting@d4086aaf8a36ae830f1714aad585900d24ad1156
- git+https://github.com/iPERDance/neural_renderer.git@e5f54f71a8941acf372514eb92e289872f272653

# 1. Installation


## 1.1 Instal ffmpeg (ffprobe) and set CUDA_HOME to the system enviroments

In [ ]:
# Install ffmpeg (ffprobe)
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
# set CUDA_HOME, here we use CUDA 10.1
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"

!echo $CUDA_HOME

/usr/local/cuda-10.1


## 1.1 Clone iPERCore Github Repo

In [ ]:
!git clone https://github.com/iPERDance/iPERCore.git

Cloning into 'iPERCore'...
remote: Enumerating objects: 831, done.
remote: Counting objects: 100% (493/493), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 831 (delta 187), reused 377 (delta 164), pack-reused 338
Receiving objects: 100% (831/831), 23.08 MiB | 31.89 MiB/s, done.
Resolving deltas: 100% (253/253), done.


## 1.2 Setup iPERCore

In [ ]:
cd /content/iPERCore/

/content/iPERCore


In [ ]:
!python setup.py develop

## 1.3 Download assets
The assets contain all pre-trained models (**checkpoints.zip**), and other executable files (**executables.zip**, this one is only used for Windows. Linux ignores it), such as ffmpeg and ffprobe.

Download links: 
  - checkpoints: https://download.impersonator.org/iper_plus_plus_latest_checkpoints.zip
  - samples: https://download.impersonator.org/iper_plus_plus_latest_samples.zip

You can manually download the **checkpoints.zip**, unzip it, and mv the **checkpoints** (as well as the **samples**) to **assets** folder. 

 Otherwise, you can just run the following scripts to automaticially do these.

In [ ]:
# Download all checkpoints
#!wget -O assets/checkpoints.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLkwQyrk3W9_H7MzNA?e=rRje0G"
!wget -O assets/checkpoints.zip "https://download.impersonator.org/iper_plus_plus_latest_checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/

!rm assets/checkpoints.zip

In [ ]:
# download samples
# !wget -O assets/samples.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLobQPpoxo2hfhURrA?e=EUyIC2"
!wget -O assets/samples.zip  "https://download.impersonator.org/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

# Run Motion Imitation Demo

In [ ]:
cd /content/iPERCore/

/content/iPERCore


In [ ]:
import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode

## Details of Config
 - gpu_ids (str): the gpu_ids, default is "0";
 - image_size (int): the image size, default is 512;
 - num_source (int): the number of source images for Attention, default is 2. Large needs more GPU memory;
 - assets_dir (str): the assets directory. This is very important, and there are the configurations and all pre-trained checkpoints;
 - output_dir (str): the output directory;

 - src_path (str): the source input information. 
       All source paths and it supports multiple paths, uses "|" as the separator between all paths. 
       The format is "src_path_1|src_path_2|src_path_3". 
       
       Each src_input is "path?=path1,name?=name1,bg_path?=bg_path1". 
       
       It must contain 'path'. If 'name' and 'bg_path' are empty, they will be ignored.

       The 'path' could be an image path, a path of a directory contains source images, and a video path.

       The 'name' is the rename of this source input, if it is empty, we will ignore it, and use the filename of the path.

       The 'bg_path' is the actual background path if provided, otherwise we will ignore it.
       
       There are several examples of formated source paths,

        1. "path?=path1,name?=name1,bg_path?=bg_path1|path?=path2,name?=name2,bg_path?=bg_path2",
        this input will be parsed as [{path: path1, name: name1, bg_path:bg_path1},
        {path: path2, name: name2, bg_path: bg_path2}];

        2. "path?=path1,name?=name1|path?=path2,name?=name2", this input will be parsed as
        [{path: path1, name:name1}, {path: path2, name: name2}];

        3. "path?=path1", this input will be parsed as [{path: path1}].

        4. "path1", this will be parsed as [{path: path1}].

 - ref_path (str): the reference input information.
       
       All reference paths. It supports multiple paths, and uses "|" as the separator between all paths.
       The format is "ref_path_1|ref_path_2|ref_path_3".

       Each ref_path is "path?=path1,name?=name1,audio?=audio_path1,fps?=30,pose_fc?=300,cam_fc?=150".

       It must contain 'path', and others could be empty, and they will be ignored.

       The 'path' could be an image path, a path of a directory contains images of a same person, and a video path.

       The 'name' is the rename of this source input, if it is empty, we will ignore it, and use the filename of the path.

       The 'audio' is the audio path, if it is empty, we will ignore it. If the 'path' is a video,
        you can ignore this, and we will firstly extract the audio information of this video (if it has audio channel).

       The 'fps' is fps of the final outputs, if it is empty, we will set it as the default fps 25.

       The 'pose_fc' is the smooth factor of the temporal poses. The smaller of this value, the smoother of the temporal poses. If it is empty, we will set it as the default 300. In the most cases, using the default 300 is enough, and if you find the poses of the outputs are not stable, you can decrease this value. Otherwise, if you find the poses of the outputs are over stable, you can increase this value.

       The 'cam_fc' is the smooth factor of the temporal cameras (locations in the image space). The smaller of this value, the smoother of the locations in sequences. If it is empty, we will set it as the default 150. In the most cases, the default 150 is enough.

       There are several examples of formated reference paths,

        1. "path?=path1,name?=name1,audio?=audio_path1,fps?=30,pose_fc?=300,cam_fc?=150|
            path?=path2,name?=name2,audio?=audio_path2,fps?=25,pose_fc?=450,cam_fc?=200",
            this input will be parsed as
            [{path: path1, name: name1, audio: audio_path1, fps: 30, pose_fc: 300, cam_fc: 150},
             {path: path2, name: name2, audio: audio_path2, fps: 25, pose_fc: 450, cam_fc: 200}]

        2. "path?=path1,name?=name1, pose_fc?=450|path?=path2,name?=name2", this input will be parsed as
        [{path: path1, name: name1, fps: 25, pose_fc: 450, cam_fc: 150},
         {path: path2, name: name2, fps: 25, pose_fc: 300, cam_fc: 150}].

        3. "path?=path1|path?=path2", this input will be parsed as
        [{path: path1, fps:25, pose_fc: 300, cam_fc: 150}, {path: path2, fps: 25, pose_fc: 300, cam_fc: 150}].

        4. "path1|path2", this input will be parsed as
        [{path: path1, fps:25, pose_fc: 300, cam_fc: 150}, {path: path2, fps: 25, pose_fc: 300, cam_fc: 150}].

        5. "path1", this will be parsed as [{path: path1, fps: 25, pose_fc: 300, cam_fc: 150}].

## Run Scripts

In [ ]:
# the gpu ids
gpu_ids = "0"

# the image size
image_size = 512

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 2

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"

# the model id of this case. This is a random model name.
# model_id = "model_" + str(time.time())

# # This is a specific model name, and it will be used if you do not change it.
# model_id = "axing_1"

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")


### Run the `trump` case
In this case, there is only a frontal body image as the source inputs.

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "donald_trump_2"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/donald_trump_2/00000.PNG,name?=donald_trump_2\""


## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
# ref_path = "\"path?=/content/iPERCore/assets/samples/references/akun_1.mp4," \
#              "name?=akun_2," \
#              "pose_fc?=300\""

ref_path = "\"path?=/content/iPERCore/assets/samples/references/mabaoguo_short.mp4," \
             "name?=mabaoguo_short," \
             "pose_fc?=400\""

# ref_path = "\"path?=/content/iPERCore/assets/samples/references/akun_1.mp4,"  \
#              "name?=akun_2," \
#              "pose_fc?=300|" \
#              "path?=/content/iPERCore/assets/samples/references/mabaoguo_short.mp4," \
#              "name?=mabaoguo_short," \
#              "pose_fc?=400\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

"path?=/content/iPERCore/assets/samples/references/akun_1.mp4,name?=akun_2,pose_fc?=300|path?=/content/iPERCore/assets/samples/references/mabaoguo_short.mp4,name?=mabaoguo_short,pose_fc?=400"
ffmpeg -y -i /content/iPERCore/assets/samples/references/akun_1.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/akun_2/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/akun_1.mp4
ffmpeg -y -i /content/iPERCore/assets/samples/references/mabaoguo_short.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/mabaoguo_short/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/mabaoguo_short.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/so

In [ ]:
mp4 = open("./results/primitives/donald_trump_2/synthesis/imitations/donald_trump_2-akun_2.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
mp4 = open("./results/primitives/donald_trump_2/synthesis/imitations/donald_trump_2-mabaoguo_short.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

This cell output is too large and can only be displayed while logged in.


### Run the `axing_1` case
In this case, there are two source images. The one is the frontal image, and the other is the backside image.

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "axing_1"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/axing_1,name?=axing_1\""


## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
ref_path = "\"path?=/content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4," \
             "name?=bantangzhuyi_1," \
             "pose_fc?=300\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

"path?=/content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4,name?=bantangzhuyi_1,pose_fc?=300"
ffmpeg -y -i /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/bantangzhuyi_1/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/sources/axing_1
	bg_path: 
	name: axing_1
primitives_dir: ./results/primitives/axing_1
processed_dir: ./results/primitives/axing_1/processed
vid_info_path: ./results/primitives/axing_1/processed/vid_info.pkl
-------------------------------------------------------
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4
	bg_path: 


In [ ]:
mp4 = open("./results/primitives/axing_1/synthesis/imitations/axing_1-bantangzhuyi_1.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

This cell output is too large and can only be displayed while logged in.


### Run the `afan_6` case
In this case, there are two source images, as well as a real background image.

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "afan_6=ns=2"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/afan_6/afan_6=ns=2," \
             "name?=afan_6=ns=2," \
             "bg_path?=/content/iPERCore/assets/samples/sources/afan_6/IMG_7217.JPG\""

## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
ref_path = "\"path?=/content/iPERCore/assets/samples/references/akGexYZug2Q_2.mp4.mp4," \
             "name?=akGexYZug2Q_2," \
             "pose_fc?=300\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

"path?=/content/iPERCore/assets/samples/references/akGexYZug2Q_2.mp4.mp4,name?=akGexYZug2Q_2,pose_fc?=300"
ffmpeg -y -i /content/iPERCore/assets/samples/references/akGexYZug2Q_2.mp4.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/akGexYZug2Q_2/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/akGexYZug2Q_2.mp4.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/sources/afan_6/afan_6=ns=2
	bg_path: /content/iPERCore/assets/samples/sources/afan_6/IMG_7217.JPG
	name: afan_6=ns=2
primitives_dir: ./results/primitives/afan_6=ns=2
processed_dir: ./results/primitives/afan_6=ns=2/processed
vid_info_path: ./results/primitives/afan_6=ns=2/processed/vid_info.pkl
-------------------------------------------------------
----------------------MetaProcess----------------------

In [ ]:
mp4 = open("./results/primitives/afan_6=ns=2/synthesis/imitations/afan_6=ns=2-akGexYZug2Q_2.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

This cell output is too large and can only be displayed while logged in.


### Run your custom inputs
You can upload your own custom source images, and reference videos with the followings guidelines.

#### Source Guidelines:
 - Try to capture the source images with the same static background without too complex scene structures. If possible, we recommend using the
actual background.
 - The person in the source images holds an A-pose for introducing the most visible textures.
 - It is recommended to capture the source images in an environment without too much contrast in lighting conditions and lock auto-exposure and auto-focus of the camera.

#### Reference Guidelines:
  - Make sure that there is only **one** person in the reference video. Since,currently, our system does not support multiple people tracking. If there are multiple people, you need firstly use other video processing tools to crop the video.
  - Make sure that capture the video with full body person. Half body will result in bad results.
  - Try to capture the video with the static camera lens, and make sure that there is no too much zoom-in, zoom-out, panning, lens swichtings, and camera transitions. If there are multiple lens switchting and camera transitions, you need firstly use other video processing tools to crop the video.

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "Name it"

# the source input information, here \" is escape character of double duote "
# src_path = "\"path?=/content/iPERCore/assets/samples/sources/afan_6/afan_6=ns=2," \
#              "name?=afan_6=ns=2," \
#              "bg_path?=/content/iPERCore/assets/samples/sources/afan_6/IMG_7217.JPG\""

src_path = "\"Make your own inputs\""

## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
# ref_path = "\"path?=/content/iPERCore/assets/samples/references/akGexYZug2Q_2.mp4.mp4," \
#              "name?=akGexYZug2Q_2," \
#              "pose_fc?=300\""

ref_path = "\"Make your own inputs\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

# Run Novel View Synthesis

## Render Novel View with T-pose

In [ ]:
!python demo/novel_view.py --gpu_ids 0 \
   --image_size 512 \
   --num_source 2   \
   --output_dir "./results" \
   --assets_dir "./assets"  \
   --model_id   "afan_6=ns=2" \
   --src_path   "path?=./assets/samples/sources/afan_6/afan_6=ns=2,name?=afan_6=ns=2,bg_path?=./assets/samples/sources/afan_6/IMG_7217.JPG" \
   --T_pose